In [2]:
pip install pyttsx3

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install gTTS


In [4]:
pip install gtts

In [7]:
import re
import pandas as pd
import pyttsx3
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier,_tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



training = pd.read_csv('/content/drive/MyDrive/ML_chatbot/Training.csv')
testing= pd.read_csv('/content/drive/MyDrive/ML_chatbot/Testing.csv')
cols= training.columns
cols= cols[:-1]
x = training[cols]
y = training['prognosis']
y1= y


reduced_data = training.groupby(training['prognosis']).max()

#mapping strings to numbers
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
testx    = testing[cols]
testy    = testing['prognosis']
testy    = le.transform(testy)


clf1  = DecisionTreeClassifier()
clf = clf1.fit(x_train,y_train)
# print(clf.score(x_train,y_train))hi

# print ("cross result========")
scores = cross_val_score(clf, x_test, y_test, cv=3)
# print ( scores)
print ("Accuracy for Decision Tree classifier: ",scores.mean())


model=SVC()
model.fit(x_train,y_train)
print("Accuracy for svm: ")

print(model.score(x_test,y_test))

importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]
features = cols


severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

symptoms_dict = {}

for index, symptom in enumerate(x):
       symptoms_dict[symptom] = index
def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp)+1)>13):
        print("You should take the consultation from doctor. ")
    else:
        print("It might not be that bad but you should take precautions.")


def getDescription():
    global description_list
    with open('/content/drive/MyDrive/ML_chatbot/symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)




def getSeverityDict():
    global severityDictionary
    with open('/content/drive/MyDrive/ML_chatbot/Symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('/content/drive/MyDrive/ML_chatbot/symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4],row[5],row[6]]}
            precautionDictionary.update(_prec)


Accuracy for Decision Tree classifier:  0.9735251788747092
Accuracy for svm: 
1.0


In [9]:
from gtts import gTTS
from IPython.display import Audio
import time

def getInfo():
    tts = gTTS(text="Hi, I am MedBot", lang='en')
    tts.save("medbot_intro.mp3")
    display(Audio("medbot_intro.mp3", autoplay=True))
    time.sleep(3)

    tts = gTTS(text="Your Name?", lang='en')
    tts.save("your_name.mp3")
    display(Audio("your_name.mp3", autoplay=True))
    time.sleep(3)

    tts = gTTS(text="Please enter your name:", lang='en')
    tts.save("enter_name.mp3")
    display(Audio("enter_name.mp3", autoplay=True))
    time.sleep(3)

    name = input("Your Name? -> ")

    tts = gTTS(text=f"Hello, {name}", lang='en')
    tts.save("hello_name.mp3")
    display(Audio("hello_name.mp3", autoplay=True))
    time.sleep(3)
# Call the getInfo function


from gtts import gTTS
from IPython.display import Audio
import time

def check_pattern(dis_list, inp):
    pred_list = []
    inp = inp.replace(' ', '_')
    patt = f"{inp}"
    regexp = re.compile(patt)
    pred_list = [item for item in dis_list if regexp.search(item)]
    if len(pred_list) > 0:
        return 1, pred_list
    else:
        return 0, []

def sec_predict(symptoms_exp):
    df = pd.read_csv('/content/drive/MyDrive/ML_chatbot/Training.csv')
    X = df.iloc[:, :-1]
    y = df['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    rf_clf = DecisionTreeClassifier()
    rf_clf.fit(X_train, y_train)

    symptoms_dict = {symptom: index for index, symptom in enumerate(X)}
    input_vector = np.zeros(len(symptoms_dict))
    for item in symptoms_exp:
      input_vector[[symptoms_dict[item]]] = 1

    return rf_clf.predict([input_vector])

def print_disease(node):
    node = node[0]
    val = node.nonzero()
    disease = le.inverse_transform(val[0])
    return list(map(lambda x: x.strip(), list(disease)))

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    chk_dis = ",".join(feature_names).split(",")
    symptoms_present = []



    while True:
        print("\nEnter the symptom you are experiencing  \t\t", end="->")
        tts = gTTS(text="Please enter the symptom you are experiencing:", lang='en')
        tts.save("enter_symptom.mp3")
        display(Audio("enter_symptom.mp3", autoplay=True))
        time.sleep(2)

        disease_input = input("")

        conf, cnf_dis = check_pattern(chk_dis, disease_input)
        if conf == 1:
            print("searches related to input: ")
            tts = gTTS(text=" please know that you're not alone. We're here to support you through this difficult time:", lang='en')
            tts.save("related_input.mp3")
            display(Audio("related_input.mp3", autoplay=True))
            time.sleep(5)

            for num, it in enumerate(cnf_dis):
                print(num, ")", it)

            if num != 0:
                print(f"Select the one you meant (0 - {num}):  ", end="")
                tts = gTTS(text=f"Select the one you meant (0 - {num}):", lang='en')
                tts.save("select_one.mp3")
                display(Audio("select_one.mp3", autoplay=True))
                time.sleep(2)

                conf_inp = int(input(""))
            else:
                conf_inp = 0

            disease_input = cnf_dis[conf_inp]
            break
        else:
            print("Enter valid symptom.")
            tts = gTTS(text="Enter valid symptom.", lang='en')
            tts.save("valid_symptom.mp3")
            display(Audio("valid_symptom.mp3", autoplay=True))
            time.sleep(2)

    while True:
        try:
            tts = gTTS(text="Okay. From how many days ?", lang='en')
            tts.save("days.mp3")
            display(Audio("days.mp3", autoplay=True))
            time.sleep(2)
            num_days = int(input("Okay. From how many days ? : "))
            break
        except:
            print("Enter valid input.")
            gTTS(text="Enter valid input.", lang='en').save("valid_input.mp3")
            display(Audio("valid_input.mp3", autoplay=True))
            time.sleep(2)

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]

            if name == disease_input:
                val = 1
            else:
                val = 0
            if val <= threshold:
                recurse(tree_.children_left[node], depth + 1)
            else:
                symptoms_present.append(name)
                recurse(tree_.children_right[node], depth + 1)
        else:
            present_disease = print_disease(tree_.value[node])


            red_cols = reduced_data.columns
            symptoms_given = red_cols[reduced_data.loc[present_disease].values[0].nonzero()]

            tts = gTTS(text="Are you experiencing any", lang='en')
            tts.save("experiencing_any.mp3")
            display(Audio("experiencing_any.mp3", autoplay=True))
            time.sleep(2)

            symptoms_exp = []
            for syms in list(symptoms_given):
                inp = ""
                tts = gTTS(text=f"{syms}?", lang='en')
                tts.save(f"{syms}_question.mp3")
                display(Audio(f"{syms}_question.mp3", autoplay=True))
                time.sleep(2)

                while True:
                    inp = input("")
                    if inp == "yes" or inp == "no":
                        break
                    else:
                        tts = gTTS(text="provide proper answers. that is (yes/no) :", lang='en')
                        tts.save("proper_answers.mp3")
                        display(Audio("proper_answers.mp3", autoplay=True))
                        time.sleep(2)

                if inp == "yes":
                    symptoms_exp.append(syms)

            second_prediction = sec_predict(symptoms_exp)
            calc_condition(symptoms_exp, num_days)
            if present_disease[0] == second_prediction[0]:
                tts = gTTS(text=f"You may have {present_disease[0]}", lang='en')
                tts.save("may_have.mp3")
                display(Audio("may_have.mp3", autoplay=True))
                time.sleep(2)

                tts = gTTS(text=description_list[present_disease[0]], lang='en')
                tts.save("disease_description.mp3")
                display(Audio("disease_description.mp3", autoplay=True))
                time.sleep(20)
            else:
                tts = gTTS(text=f"You may have {present_disease[0]} or {second_prediction[0]}", lang='en')
                tts.save("may_have_multiple.mp3")
                display(Audio("may_have_multiple.mp3", autoplay=True))
                time.sleep(4)

                tts = gTTS(text=description_list[present_disease[0]], lang='en')
                tts.save("disease_description.mp3")
                display(Audio("disease_description.mp3", autoplay=True))
                time.sleep(20)

                tts = gTTS(text=description_list[second_prediction[0]], lang='en')
                tts.save("second_disease_description.mp3")
                display(Audio("second_disease_description.mp3", autoplay=True))
                time.sleep(20)

            precution_list = precautionDictionary[present_disease[0]]
            tts = gTTS(text="Take following measures:", lang='en')
            tts.save("take_measures.mp3")
            display(Audio("take_measures.mp3", autoplay=True))
            time.sleep(4)

            for i, j in enumerate(precution_list):
                tts = gTTS(text=f"{i+1}) {j}", lang='en')
                tts.save(f"precaution_{i}.mp3")
                display(Audio(f"precaution_{i}.mp3", autoplay=True))
                time.sleep(4)


    recurse(0, 1)

# Call your functions
getSeverityDict()
getDescription()
getprecautionDict()
getInfo()
tree_to_code(clf, cols)
print("----------------------------------------------------------------------------------------")


Your Name? -> a



Enter the symptom you are experiencing  		->

KeyboardInterrupt: Interrupted by user

In [ ]:
ai